
구현 참고: https://github.com/gbolmier/funk-svd/blob/master/funk_svd/fast_methods.py


In [2]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../src")

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
ds_ratings = pd.read_csv("../ml-latest-small/ratings.csv")
ds_movies = pd.read_csv("../ml-latest-small/movies.csv")
ds_ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [5]:
seed = 1 # 
# 유저별 평가 개수를 얻어옵니다
rating_counts = ds_ratings.groupby("userId").count().movieId
# 100회 이상 평가한 유저 10명을 무작위로 선발합니다
random_users = rating_counts[rating_counts >= 100].sample(10, random_state=seed)
dropped_ratings = []

for user_id in random_users.index:
    user_ratings = ds_ratings[ds_ratings.userId == user_id]
    
    # 4점 이상 평가 5개, 4점 미만 평가 4개를 무작위로 선정합니다.
    dropped_ratings.append(
        user_ratings[user_ratings.rating >= 4].sample(5, random_state=seed)
    )
    dropped_ratings.append(
        user_ratings[user_ratings.rating < 4].sample(5, random_state=seed)
    )
    
dropped_ratings = pd.concat(dropped_ratings)
dropped_ratings

,userId,movieId,rating,timestamp
22513,153,8784,4.5,1525548730
22525,153,55247,5.0,1525548301
22458,153,1704,4.0,1525548099
22571,153,109374,4.0,1525548490
22593,153,168492,4.5,1525548280
...,...,...,...,...
1207,10,72330,3.5,1455399573
1235,10,95543,3.5,1455399376
1187,10,56367,3.5,1455301825
1203,10,70183,3.0,1455357636


In [6]:
# 선정된 평가들 100개는 학습에서 제외시킵니다
ds_ratings.drop(dropped_ratings.index, inplace=True)
ds_ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [121]:
learning_rate = 0.01
num_epochs = 100
regularization = 0.01
k = 64

# len(ds_movies)
len(ds_movies.movieId.unique())

9742

In [148]:
def normalize(X, min_value, max_value):
    return (X - X.min()) / (X.max() - X.min()) * (max_value - min_value) + min_value

class FunkSVD:
    def __init__(self, ratings, movies, k, regularization=0.2):
        self.ratings = ratings.copy()
        self.ratings.rating = normalize(self.ratings.rating, 0, 1)
        self.movies = movies
        self.global_mean = self.ratings.rating.mean()
        self.regularization = regularization
        
        # 유저, 아이템 ID가 인덱스와 일치하지 않으므로 변환하기 위한 사전
        self.user_ids = {user: i for i, user in enumerate(ratings.userId.unique())}
        self.item_ids = {item: i for i, item in enumerate(movies.movieId.unique())}
        
        self.num_users = len(self.user_ids)
        self.num_items = len(self.item_ids)
        
        # 유저 행렬(N x k), 아이템 행렬(M x k)
        self.users = np.random.normal(0.0, 1.0, (self.num_users, k))
        self.items = np.random.normal(0.0, 1.0, (self.num_items, k))
        
        # 편향 벡터들(N x 1, M x 1)
        self.user_biases = np.zeros(self.num_users)
        self.item_biases = np.zeros(self.num_items)

    def fit(self, num_epochs=100, learning_rate=0.005):
        for epoch in range(num_epochs):
            total_error = 0
            
            for i, (user_id, item_id, rating, timestamp) in self.ratings.iterrows():
                # 유저 벡터와 아이템 백터를 가져온 뒤 내적하여 예측합니다
                user_id = self.user_ids[user_id]
                item_id = self.item_ids[item_id]

                pred = np.dot(self.users[user_id], self.items[item_id]) + self.global_mean
                pred += self.user_biases[user_id] + self.item_biases[item_id]

                # 예측값과 실제 값의 차이(error)를 계산하고
                # learning rate만큼 갱신합니다.
                error = rating - pred
                
                self.user_biases[user_id] += learning_rate * (error -  self.regularization * self.user_biases[user_id])
                self.item_biases[item_id] += learning_rate * (error -  self.regularization * self.item_biases[item_id])
                
                self.users[user_id] += learning_rate * (error * self.items[item_id] - self.regularization * self.users[user_id])
                self.items[item_id] += learning_rate * (error * self.users[user_id] - self.regularization * self.items[item_id]) 

                total_error += error ** 2

            total_error /= len(self.ratings)

            print("epoch {0}, mean_error: {1}".format(epoch, total_error))
            
        return self
    
    def get_recommendations(self, user_id, k=None):
        user_movie_ids = self.ratings[self.ratings.userId == user_id].movieId
        
        user_row_id = self.user_ids[user_id]
        
        # 유저 행렬과 전체 아이템 행렬을 곱해서 모든 영화에 대한 예측값을 구합니다
        preds = np.dot(self.users[user_row_id], self.items.T) + self.global_mean
        preds = preds * 5
        # 예측값에서 유저가 이미 본 영화는 제외하고 점수순으로 정렬합니다.
        preds = pd.Series(preds, index=self.item_ids.keys())
        preds = preds[~preds.index.isin(user_movie_ids)]
        preds = pd.DataFrame({
            "movie_id": preds.index,
            "score": preds.values
        })
        preds.sort_values(by="score", ascending=False, inplace=True)
        preds["rank"] = range(1, len(preds) + 1)
        
        preds = preds.head(k) if k is not None else preds
    
        recommended_movies = self.movies[self.movies.movieId.isin(preds.movie_id)]
        recommended_movies = recommended_movies.merge(
            preds,
            left_on="movieId",
            right_on="movie_id"
        ).sort_values("rank")
        
        del recommended_movies["movie_id"]
        
        return recommended_movies
        
    def predict_rating(self, user_id, item_id):
        user_row_id = self.user_ids[user_id]
        item_row_id = self.item_ids[item_id]
        
        pred = np.dot(self.users[user_row_id], self.items[item_row_id]) + self.global_mean
        
        return pred * 5

In [149]:
funksvd = FunkSVD(ds_ratings, ds_movies, k).fit(learning_rate=learning_rate, num_epochs=10)
# funksvd.global_mean

epoch 0, mean_error: 13.316329108348272
epoch 1, mean_error: 0.6029268335678294
epoch 2, mean_error: 0.121714409761506
epoch 3, mean_error: 0.06089990971195851
epoch 4, mean_error: 0.04390227050429089
epoch 5, mean_error: 0.037437681775976214
epoch 6, mean_error: 0.03448388110979064
epoch 7, mean_error: 0.032939503772051876
epoch 8, mean_error: 0.032043029773714204
epoch 9, mean_error: 0.03147906761062425


In [150]:
funksvd.get_recommendations(609, 10)

,movieId,title,genres,score,rank
7,93805,Iron Sky (2012),Action|Comedy|Sci-Fi,59.730406,1
3,8518,Anna Christie (1930),Drama,59.513044,2
8,109317,Someone Marry Barry (2014),Comedy,58.915483,3
9,166183,Junior and Karlson (1968),Adventure|Animation|Children,57.040604,4
4,51666,"Abandoned, The (2006)",Horror|Mystery|Thriller,57.016808,5
1,4082,Barfly (1987),Comedy|Drama|Romance,56.395060,6
6,61394,"Onion Movie, The (2008)",Comedy,55.116010,7
0,3125,"End of the Affair, The (1999)",Drama,53.484077,8
2,5685,Real Women Have Curves (2002),Comedy|Drama,51.630308,9
5,56389,My Blueberry Nights (2007),Drama|Romance,51.090840,10


In [151]:
funksvd.predict_rating(609, 432)

0.6015662197744598

In [126]:
from svd import SVD


svd = SVD(ds_ratings, ds_movies, 64)

In [152]:
from IPython.display import display


win_rates = 0

for user_id in dropped_ratings.userId.unique():
    targets = dropped_ratings[dropped_ratings.userId == user_id].movieId
    svd_rec = svd.get_recommendations(user_id)
    svd_rec = svd_rec[svd_rec.movieId.isin(targets)]
    svd_rec["svd_rank"] = svd_rec.index
    svd_rec.index = svd_rec.movieId
    
    funksvd_rec = funksvd.get_recommendations(user_id)
    funksvd_rec = funksvd_rec[funksvd_rec.movieId.isin(targets)]
    funksvd_rec.index = funksvd_rec.movieId
    funksvd_rec.rename({"rank":"funksvd_rank"}, axis=1, inplace=True)
    
    svd_rec.drop(["recommendation_score", "movieId"], axis=1, inplace=True)
    result = pd.concat([svd_rec, funksvd_rec["funksvd_rank"]], axis=1)
    display(result)
    
    win_rate = (result.funksvd_rank < result.svd_rank).mean()
    win_rates += win_rate
    print("funksvd > svd : ", win_rate, "rating_count", ds_ratings[ds_ratings.userId == user_id].count() + 10)

print("total", win_rates / len(dropped_ratings.userId.unique()))

../src/svd.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_recommendations["recommendation_score"] = user_predictions.values


,title,genres,svd_rank,funksvd_rank
movieId,,,,
1704,Good Will Hunting (1997),Drama|Romance,1284,5573
2671,Notting Hill (1999),Comedy|Romance,2007,5158
3408,Erin Brockovich (2000),Drama,2546,4514
8784,Garden State (2004),Comedy|Drama|Romance,5302,5468
55247,Into the Wild (2007),Action|Adventure|Drama,6580,2543
63082,Slumdog Millionaire (2008),Crime|Drama|Romance,6885,4280
69757,(500) Days of Summer (2009),Comedy|Drama|Romance,7075,3808
72998,Avatar (2009),Action|Adventure|Sci-Fi|IMAX,7212,5353
109374,"Grand Budapest Hotel, The (2014)",Comedy|Drama,8372,5693


funksvd > svd :  0.6 rating_count userId       179
movieId      179
rating       179
timestamp    179
dtype: int64


../src/svd.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_recommendations["recommendation_score"] = user_predictions.values


,title,genres,svd_rank,funksvd_rank
movieId,,,,
968,Night of the Living Dead (1968),Horror|Sci-Fi|Thriller,741.0,7420
1343,Cape Fear (1991),Thriller,1031.0,6742
2716,Ghostbusters (a.k.a. Ghost Busters) (1984),Action|Comedy|Sci-Fi,2038.0,4730
3147,"Green Mile, The (1999)",Crime|Drama,2372.0,4235
3727,Near Dark (1987),Horror|Western,2786.0,3167
8865,Sky Captain and the World of Tomorrow (2004),Action|Adventure|Sci-Fi,5330.0,7081
60941,"Midnight Meat Train, The (2008)",Horror|Mystery|Thriller,6812.0,2011
94864,Prometheus (2012),Action|Horror|Sci-Fi|IMAX,7888.0,802
107771,NaN,NaN,NaN,582


...

,title,genres,svd_rank,funksvd_rank
movieId,,,,
317,"Santa Clause, The (1994)",Comedy|Drama|Fantasy,276,5294
355,"Flintstones, The (1994)",Children|Comedy|Fantasy,313,6929
415,Another Stakeout (1993),Comedy|Thriller,359,7405
457,"Fugitive, The (1993)",Thriller,398,4898
471,"Hudsucker Proxy, The (1994)",Comedy,409,2949
508,Philadelphia (1993),Drama,443,5718
531,"Secret Garden, The (1993)",Children|Drama,464,4231
551,"Nightmare Before Christmas, The (1993)",Animation|Children|Fantasy|Musical,483,6683
589,Terminator 2: Judgment Day (1991),Action|Sci-Fi,507,4781


funksvd > svd :  0.0 rating_count userId       106
movieId      106
rating       106
timestamp    106
dtype: int64


../src/svd.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_recommendations["recommendation_score"] = user_predictions.values


,title,genres,svd_rank,funksvd_rank
movieId,,,,
1907,Mulan (1998),Adventure|Animation|Children|Comedy|Drama|Musi...,1390.0,6556
56367,Juno (2007),Comedy|Drama|Romance,6638.0,5722
68954,Up (2009),Adventure|Animation|Children|Drama,7039.0,5298
70183,"Ugly Truth, The (2009)",Comedy|Drama|Romance,7087.0,3564
72330,NaN,NaN,NaN,9038
94070,"Best Exotic Marigold Hotel, The (2011)",Comedy|Drama,7867.0,5929
95543,Ice Age 4: Continental Drift (2012),Adventure|Animation|Comedy,7929.0,8042
96079,Skyfall (2012),Action|Adventure|Thriller|IMAX,7955.0,4127
103339,White House Down (2013),Action|Drama|Thriller|IMAX,8201.0,7137


funksvd > svd :  0.7 rating_count userId       140
movieId      140
rating       140
timestamp    140
dtype: int64
total 0.38999999999999996


In [128]:
from IPython.display import display


for user_id in dropped_ratings.userId.unique():
    ratings = dropped_ratings[dropped_ratings.userId == user_id] 
    rec = funksvd.get_recommendations(user_id)
    rec = rec[rec.movieId.isin(ratings.movieId)]
    
    rec = ratings.merge(rec, left_on="movieId", right_on="movieId")
    rec.sort_values(by="rating", ascending=False, inplace=True)
    rec["diff"] = rec.rating - rec.score
    display(rec)
    
    break

,userId,movieId,rating,timestamp,title,genres,score,rank,diff
1,153,55247,5.0,1525548301,Into the Wild (2007),Action|Adventure|Drama,2.801153,7331,2.198847
0,153,8784,4.5,1525548730,Garden State (2004),Comedy|Drama|Romance,3.575066,4452,0.924934
4,153,168492,4.5,1525548280,Call Me by Your Name (2017),Drama|Romance,3.888095,3243,0.611905
2,153,1704,4.0,1525548099,Good Will Hunting (1997),Drama|Romance,3.400796,5151,0.599204
3,153,109374,4.0,1525548490,"Grand Budapest Hotel, The (2014)",Comedy|Drama,3.689779,3985,0.310221
6,153,69757,3.5,1525548722,(500) Days of Summer (2009),Comedy|Drama|Romance,3.947745,3017,-0.447745
9,153,3408,2.5,1525548836,Erin Brockovich (2000),Drama,3.260822,5708,-0.760822
7,153,63082,1.5,1525548393,Slumdog Millionaire (2008),Crime|Drama|Romance,3.508330,4712,-2.008330
5,153,72998,1.0,1525552980,Avatar (2009),Action|Adventure|Sci-Fi|IMAX,3.769540,3660,-2.769540
8,153,2671,1.0,1525550835,Notting Hill (1999),Comedy|Romance,2.091108,8791,-1.091108
